# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

1. Face Detection : 이미지나 동영상에서 얼굴을 찾는 것
2. Face Recognition : 그 얼굴이 누구의 얼굴인지 알아보는 것

패키지 설치
> pip install mediapipe

예제 동영상 : https://www.pexels.com/video/3256542/  
크기 : Full HD (1920 x 1080)  
파일명 : face_video.mp4

캐릭터 이미지 : https://www.freepik.com/free-vector/cute-animal-masks-video-chat-application-effect-filters-set_6380101.htm  
파일명 : right_eye.png (100 x 100), left_eye.png (100 x 100), nose.png (300 x 100)

무료 이미지 편집 도구 : https://pixlr.com/kr/  
(Pixlr E -Advanced Editor)

## 의도
: 동영상파일에서 사람 얼굴을 하나 찾아서 거기에 캐릭터 모양 씌워주기

### 그냥 얼굴인식 + 간단한 동그라미들로 눈코 위치 파악하기

In [1]:
# 처음에 그냥 얼굴인식 + 간단한 동그라미들로 눈코 위치 파악하기

import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
#min_detection_confidence : 70%의 확률로 이건 사람인 것 같을 때 인식 -> 높일 수록 정확도가 높아짐
#model_selection은 0/1이렇게 올 수 있는데 0은 근접한 사람 위주로 detect해줌
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
          
         # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 -> 이런 특징을 이 순서대로 뽑아줌
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분
                
                #h,w를 왜 곱하냐면 keypoints는 화면 상 어디에 있다는 비율을 나타내는거라서 곱해줘야함
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                
                 #양 눈에 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색                
                #코에 동그라미 그리기
                cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA) # 노란색
                
 # 동영상 크기 줄이기
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

### 그림판 이미지 가져와서 눈코입 위치에 씌우기

In [ ]:
#그림판 이미지 가져와서 눈코입 위치에 씌우기

import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기 <<new!!
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100 (가로, 세로)


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
#min_detection_confidence : 70%의 확률로 이건 사람인 것 같을 때 인식 -> 높일 수록 정확도가 높아짐
#model_selection은 0/1이렇게 올 수 있는데 0은 근접한 사람 위주로 detect해줌
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
          
         # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 -> 이런 특징을 이 순서대로 뽑아줌
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분
                
                
                #h,w를 왜 곱하냐면 keypoints는 화면 상 어디에 있다는 비율을 나타내는거라서 곱해줘야함
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                #이미지는 세로, 가로 영역을 지정해줘야함 << new!!!
                # 각 특징에다가 이미지 그리기                
                image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye #눈에 넣을 이미지의 크기가 100인데 right_eye의 위치 기준으로 양옆으로 50씩 벌린만큼이 크기라는 뜻
                #[right_eye[1]-50 오른쪽 눈 y좌표 기준 위로 50 아래로 50
                image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
               
                
                
 # 동영상 크기 줄이기
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

### 캐릭터 이미지 씌우기

이미지 투명도가 잘 가져와지는지 확인하기 -> unchanged 쓰면 잘 가져와짐

In [ ]:
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) 

In [ ]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용


#함수는 3차원인 동영상에 투명도까지 합한 4차원이미지를 씌울 수 없어 4차원에서 원하는 부분만 골라서 가져오기 위함임
def overlay(image, x, y, w, h, overlay_image): # 대상 이미지 (3채널), x, y 좌표, width, height, 덮어씌울 이미지 (4채널)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 255 로 나누면 0 ~ 1 사이의 값 (1: 불투명, 0: 완전)
    # (255, 255)  ->  (1, 1)
    # (255, 0)        (1, 0)
    
    # 1 - mask_image ?
    # (0, 0) -> 1- (1,1)
    # (0, 1) -> 1-(1,0)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
        #[세로영역, 가로영역, 채널은 하나씩 증가하는 c넣기]

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기 <<new!!
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100 (가로, 세로)


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
#min_detection_confidence : 70%의 확률로 이건 사람인 것 같을 때 인식 -> 높일 수록 정확도가 높아짐
#model_selection은 0/1이렇게 올 수 있는데 0은 근접한 사람 위주로 detect해줌
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
          
         # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 -> 이런 특징을 이 순서대로 뽑아줌
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분
                
                
                #h,w를 왜 곱하냐면 keypoints는 화면 상 어디에 있다는 비율을 나타내는거라서 곱해줘야함
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
               
                
                
 # 동영상 크기 줄이기
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()